
# 作業: 
    
    (1)以, Adam, 為例, 調整 batch_size, epoch , 觀察accurancy, loss 的變化
    
    (2)以同一模型, 分別驗證 SGD, Adam, Rmsprop 的 accurancy

In [1]:
import tensorflow.keras
#from keras.datasets import cifar10
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy 


In [2]:
#    第一步：選擇模型, 順序模型是多個網絡層的線性堆疊
 
model = Sequential()

#   第二步：構建網絡層
 
model.add(Dense( 500,input_shape=(784,))) # 輸入層，28*28=784   
model.add(Activation('relu')) # 激活函數是relu   

model.add(Dense( 500)) # 隱藏層節點500個   
model.add(Activation('relu'))  

model.add(Dense( 500)) # 隱藏層節點500個   
model.add(Activation('relu'))  

model.add(Dense( 500)) # 隱藏層節點500個   
model.add(Activation('relu'))  

model.add(Dense( 10)) # 輸出結果是10個類別，所以維度是10   
model.add(Activation('softmax')) # 最後一層用softmax作為激活函數

In [3]:
# 模型建立完成後，統計參數總量
print("Total Parameters：%d" % model.count_params())

Total Parameters：1149010


In [4]:
# 輸出模型摘要資訊
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               392500    
_________________________________________________________________
activation (Activation)      (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_1 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_2 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               2

In [5]:
 '''
 SGD(隨機梯度下降) - Arguments
lr: float >= 0. Learning rate.
momentum: float >= 0. Parameter that accelerates SGD in the relevant direction and dampens oscillations.
decay: float >= 0. Learning rate decay over each update.
nesterov: boolean. Whether to apply Nesterov momentum.
'''

'''
RMSprop- Arguments
lr: float >= 0. Learning rate.
rho: float >= 0.
epsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().
decay: float >= 0. Learning rate decay over each update.
'''


'\nRMSprop- Arguments\nlr: float >= 0. Learning rate.\nrho: float >= 0.\nepsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().\ndecay: float >= 0. Learning rate decay over each update.\n'

In [6]:
#  第四步：資料分割
# 使用Keras自帶的mnist工具讀取數據（第一次需要聯網）
(X_train, y_train), (X_test, y_test) = mnist.load_data() 

# 由於mist的輸入數據維度是(num, 28 , 28)，這裡需要把後面的維度直接拼起來變成784維   
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2 ])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2 ])  
Y_train = (numpy.arange(10) == y_train[:, None]).astype(int)
Y_test = (numpy.arange(10) == y_test[:, None]).astype(int)


In [7]:
'''
   宣告並設定
   batch_size：對總的樣本數進行分組，每組包含的樣本數量
   epochs ：訓練次數
   
''' 
batch_size_list = [256, 512]
epochs_list = [5, 10]

In [8]:
# 第五步：訓練, 修正 model 參數
#Blas GEMM launch failed , 避免動態分配GPU / CPU, 出現問題
import tensorflow as tf
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
#sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# 第三步：編譯, 
opt = optimizers.Adam()

for batch_size in batch_size_list:
    for epochs in epochs_list:
        model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
        history = model.fit(X_train,Y_train,batch_size = batch_size, epochs=epochs, shuffle=True,verbose=1,validation_split=0.3 )
        
        #    第六步：輸出
        print ( "" )
        print ( " test set " )
        scores = model.evaluate(X_test,Y_test,batch_size=200,verbose= 0)
        print ( " The batch size=%f, epochs=%f" %(batch_size, epochs) )
        #print ( " The test loss is %f " % scores)
        print ( " The test loss is %f " %scores[0])
        result = model.predict(X_test,batch_size=200,verbose= 0)

        result_max = numpy.argmax(result, axis = 1 )
        test_max = numpy.argmax(Y_test, axis = 1 )

        result_bool = numpy.equal(result_max, test_max)
        true_num = numpy.sum(result_bool)
        print ( " The accuracy of the model is %f " % (true_num/len(result_bool)))

Train on 42000 samples, validate on 18000 samples
Epoch 1/5
42000/42000 [==============================] - 5s 124us/sample - loss: 2.3197 - accuracy: 0.8481 - val_loss: 2.4398 - val_accuracy: 0.8413
Epoch 2/5
42000/42000 [==============================] - 4s 99us/sample - loss: 2.4327 - accuracy: 0.8418 - val_loss: 2.4438 - val_accuracy: 0.8411
Epoch 3/5
42000/42000 [==============================] - 5s 114us/sample - loss: 2.5719 - accuracy: 0.8328 - val_loss: 2.7452 - val_accuracy: 0.8215
Epoch 4/5
42000/42000 [==============================] - 6s 147us/sample - loss: 2.7587 - accuracy: 0.8206 - val_loss: 2.7452 - val_accuracy: 0.8215
Epoch 5/5
42000/42000 [==============================] - 6s 139us/sample - loss: 2.7587 - accuracy: 0.8206 - val_loss: 2.7452 - val_accuracy: 0.8215

 test set 
 The batch size=256.000000, epochs=5.000000
 The test loss is 2.759624 
 The accuracy of the model is 0.102800 
Train on 42000 samples, validate on 18000 samples
Epoch 1/10
42000/42000 [========